# Generating music

#### Importing liabraries

In [24]:
from music21 import *
import pickle
import glob
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

#### loading notes

In [25]:
with open('data/notes_pop.pickle', 'rb') as f:
    notes = pickle.load(f)

pitchNames = set (notes)
nNotes = len (pitchNames)
pitchNames = sorted (pitchNames)

#### Encoding notes

In [26]:
noteCoding = {}

code = 0
for i in pitchNames :
    noteCoding [i] = code
    code += 1

#### One hot encoding

In [27]:
sequence = 100
nInput = []
nOutput = []

for i in range (0, len (notes) - sequence, 1) : 
    
    sIn = notes [i : i + sequence]
    sOut = notes [i + sequence]
    nInput.append ([noteCoding [c]for c in sIn])
    nOutput.append (noteCoding [sOut])

In [28]:
nInput = np.reshape (nInput, (len (nInput), sequence, 1))

In [29]:
nOutput = to_categorical (nOutput)

#### Model

In [30]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(nInput.shape[1], nInput.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.25))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(nNotes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [31]:
model.load_weights('weights/pop/weights-improvement-98-0.0429-bigger.hdf5')

#### Generating random inputs

In [32]:
s = np.random.randint (0, len (notes))
inNotes = list (np.ravel (nInput[s]))

outputElements = []

noteCoding = dict((number, note) for number, note in enumerate (pitchNames))

#### Generating notes

In [35]:
for i in range (100) :
    
    arrayInput = np.array (inNotes)
    netIn = np.reshape (arrayInput, (1, sequence, 1)) / float (nNotes)
    
    prediction = np.argmax (model.predict (netIn, verbose = 1))
    
    inNotes = inNotes [1:100]
    inNotes.append (prediction)
    
    outputElements.append (noteCoding [prediction])

print (outputElements)

1/1 [==============================] - 0s 202ms/step
['E6', 'C6', 'D6', 'E6', 'C6', 'D6', 'C6', 'A6', 'G6', 'C6', 'C6', 'A6', 'G6', 'D6', 'E6', 'G6', 'G6', 'A6', 'G6', 'C6', 'E6', 'E6', 'C6', 'D6', 'E6', 'C6', 'D6', 'E6', 'C6', 'B5', 'C6', 'G6', 'E6', 'D6', 'E6', 'C6', 'C6', 'A6', 'G6', 'D6', 'C6', 'G6', 'E6', 'D6', 'E6', 'C6', 'C6', 'A6', 'G6', 'D6', 'G6', 'G6', 'C6', 'A6', 'G6', 'C6', 'E6', 'D6', 'G5', 'D6', 'E6', 'F6', 'E6', 'D6', 'C6', 'C6', 'C6', 'A6', 'G6', 'C6', 'E6', 'D6', 'E6', 'C6', 'A6', 'G6', 'C6', 'D4', 'C#4', 'D4', '7.11.2', 'D4', 'C#4', 'D4', '2.6.9', 'D4', 'C#4', 'D4', '9.1.4', 'D4', 'C#4', 'D4', '2.6', 'C#5', 'B4', 'D4', 'C#4', 'D4', 'B4', '2.7', 'D4', 'C#4', 'D4', 'A4', '2.6.9', 'D4', 'C#4', 'D4', 'A4', '1.4', 'D4', 'C#4', 'D4', '2.6', 'C#5', 'B4', 'D4', 'C#4', 'D4', '7.11.2', 'D4', 'C#4', 'D4', '2.6.9', 'D4', 'C#4', 'D4', '9.1.4', 'D4', 'C#4', 'D4', '2.6', 'C#5', 'B4', 'D4', 'C#4', 'D4', '7.11.2', 'D4', 'C#4', 'D4', '2.6.9', 'D4', 'C#4', 'D4', '9.1.4', 'D4', 'C#4', '

#### Converting notes to music file

In [36]:
outputMusic = []
offset = 0

for ele in outputElements : 
    
    if ('.' in ele) or ele.isdigit () :
        n = []
        
        for i in ele.split ('.') : 
            temp = note.Note (int (i))
            temp.storedInstrument = instrument.Piano ()
            n.append (temp)
        
        temp = chord.Chord (n)
        temp.offset = offset
        
        outputMusic.append (temp)
        
    else : 
        temp = note.Note (ele)
        temp.storedInstrument = instrument.Piano ()
        temp.offset = offset
        
        outputMusic.append (temp)
        
    offset += 0.5
        
num = 1
path = 'output/singletrack/pop/*.mid'
for i in glob.glob (path) :
    t = int (i[len (path):-4])
    if (t > num) :
        num = t
outputMusic = stream.Stream (outputMusic)
outputMusic.write ('midi', fp = 'output/singletrack/pop/music' + str (num + 1) + '.mid')

'output/singletrack/pop/music4.mid'

#### Music Generated